In [1]:
#使用神经网络
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
## 基础工具
import numpy as np
import pandas as pd
import warnings
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import jn
from IPython.display import display, clear_output
import time
import missingno as msno
#导入warnings包，利用过滤器来实现忽略警告语句。
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
#载入训练集和测试集；
path = './2sc/'
train_data = pd.read_csv(path+'used_car_train_20200313.csv', sep=' ')
test_data = pd.read_csv(path+'used_car_testB_20200421.csv', sep=' ')

In [3]:
train_data["notRepairedDamage"].replace("-", "0.0", inplace=True)
train_data["notRepairedDamage"] = train_data["notRepairedDamage"].astype("float64")

In [4]:
test_data["notRepairedDamage"].replace("-", "0.0", inplace=True)
test_data["notRepairedDamage"] = test_data["notRepairedDamage"].astype("float64")

In [5]:
train_data.fillna(train_data.median(), inplace=True)
test_data.fillna(test_data.median(), inplace=True)

In [6]:
train_data["power"].describe()

count    150000.000000
mean        119.316547
std         177.168419
min           0.000000
25%          75.000000
50%         110.000000
75%         150.000000
max       19312.000000
Name: power, dtype: float64

In [7]:
test_data["power"][test_data["power"]>600] = 600

In [8]:
#特征归一化
train_data["power"][train_data["power"]>600] = 600
features = train_data.columns.tolist()
features.remove("price")
features.remove("SaleID")
features

['name',
 'regDate',
 'model',
 'brand',
 'bodyType',
 'fuelType',
 'gearbox',
 'power',
 'kilometer',
 'notRepairedDamage',
 'regionCode',
 'seller',
 'offerType',
 'creatDate',
 'v_0',
 'v_1',
 'v_2',
 'v_3',
 'v_4',
 'v_5',
 'v_6',
 'v_7',
 'v_8',
 'v_9',
 'v_10',
 'v_11',
 'v_12',
 'v_13',
 'v_14']

In [9]:
min_max_scaler = MinMaxScaler()
x = min_max_scaler.fit_transform(train_data[features].values)
x_test = min_max_scaler.transform(test_data[features].values)

In [10]:
y = train_data["price"].values

In [11]:
from sklearn.model_selection import GridSearchCV,cross_val_score,StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2)

In [13]:
#搭建模型
model = keras.Sequential([
    keras.layers.Dense(300, activation="relu", input_shape=[len(features)]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(200, activation="relu"),
    keras.layers.Dense(1)
])
model.compile(loss="mae", optimizer="Adam")
model.fit(train_x, train_y, batch_size=1024, epochs=200)

Epoch 1/200
118/118 [==============================] - 5s 41ms/step - loss: 4865.3647
Epoch 2/200
118/118 [==============================] - 5s 41ms/step - loss: 2674.6047
Epoch 3/200
118/118 [==============================] - 5s 39ms/step - loss: 1243.4736
Epoch 4/200
118/118 [==============================] - 5s 40ms/step - loss: 1060.4109
Epoch 5/200
118/118 [==============================] - 5s 43ms/step - loss: 972.3500
Epoch 6/200
118/118 [==============================] - 5s 40ms/step - loss: 912.6943
Epoch 7/200
118/118 [==============================] - 5s 43ms/step - loss: 871.0223
Epoch 8/200
118/118 [==============================] - 5s 40ms/step - loss: 835.9439
Epoch 9/200
118/118 [==============================] - 5s 42ms/step - loss: 804.7776
Epoch 10/200
118/118 [==============================] - 5s 40ms/step - loss: 778.9166
Epoch 11/200
118/118 [==============================] - 5s 39ms/step - loss: 755.8083
Epoch 12/200
118/118 [==============================] - 5s 

In [14]:
#模型评估
print("训练集评估 MAE： ", mean_absolute_error(train_y, model.predict(train_x)))
print("测试集评估 MAE： ", mean_absolute_error(test_y, model.predict(test_x)))

训练集评估 MAE：  479.52080325080556
测试集评估 MAE：  488.9790857819875


In [15]:
#用全量数据训练
model = keras.Sequential([
    keras.layers.Dense(200, activation="relu", input_shape=[len(features)]),
    keras.layers.Dense(200, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(1)
])
model.compile(loss="mae", optimizer="Adam")
model.fit(x, y, batch_size=1024, epochs=200)

Epoch 1/200
147/147 [==============================] - 2s 16ms/step - loss: 4940.3032
Epoch 2/200
147/147 [==============================] - 2s 16ms/step - loss: 2943.2788 0s - l
Epoch 3/200
147/147 [==============================] - 2s 16ms/step - loss: 1338.6991
Epoch 4/200
147/147 [==============================] - 2s 17ms/step - loss: 1099.8873
Epoch 5/200
147/147 [==============================] - 3s 17ms/step - loss: 1010.2263
Epoch 6/200
147/147 [==============================] - 2s 16ms/step - loss: 949.4509
Epoch 7/200
147/147 [==============================] - 2s 16ms/step - loss: 901.7724
Epoch 8/200
147/147 [==============================] - 2s 16ms/step - loss: 867.7635
Epoch 9/200
147/147 [==============================] - 2s 16ms/step - loss: 837.2961: 0s - loss: 839
Epoch 10/200
147/147 [==============================] - 2s 16ms/step - loss: 813.5219
Epoch 11/200
147/147 [==============================] - 3s 17ms/step - loss: 789.6253
Epoch 12/200
147/147 [=============

In [16]:
#输出结果
predict_y = model.predict(x_test)

result = pd.DataFrame()
result["SaleID"] = test_data["SaleID"]
result["price"] = predict_y
#result["price"][result["price"] < 11] = 11
result.loc[result["price"] <11, "price"] = 11
result.loc[result["price"] >99999, "price"] = 99999
result.to_csv("ans_nn.csv", index=False)

score:493.6148

In [17]:
#定义统计函数，方便了解数据分布
def show_stats(data):
    print("min： ", np.min(data))
    print("max： ", np.max(data))
    print("ptp： ", np.ptp(data))
    print("mean： ", np.mean(data))
    print("std： ", np.std(data))
    print("var： ", np.var(data))

In [18]:
show_stats(predict_y)

min：  -995.42303
max：  110998.73
ptp：  111994.15
mean：  5828.251
std：  7371.088
var：  54332940.0


In [19]:
show_stats(result)

min：  SaleID    200000.0
price         11.0
dtype: float64
max：  SaleID    249999.0
price      99999.0
dtype: float64
ptp：  249988.0
mean：  SaleID    224999.500000
price       5828.025525
dtype: float64
std：  SaleID    14433.756727
price      7366.851290
dtype: float64
var：  SaleID    2.083333e+08
price     5.427050e+07
dtype: float64
